ASAP CRN Metadata validation - wave 1

# ASAP CRN Metadata validation - wave 1

Teams Hardy, Hafler, and Lee need some minor rationalization.


05 Nov 2023
Andy Henrie




In [1]:
import pandas as pd

from pathlib import Path

from utils.io import read_meta_table, get_dtypes_dict, ReportCollector
from utils.qcutils import prep_table, validate_table, validate_file, read_ASAP_CDE, reorder_table_to_CDE



%load_ext autoreload
%autoreload 2


Streamlit NOT successfully


## Tests

Test reading tables with non-unicode text

In [2]:
fnm = Path.home() / 'Downloads' / 'STUDY_symbols.csv'

# STUDY = pd.read_csv(fnm, index_col=0)
try:
    STUDY = pd.read_csv(fnm)
except UnicodeDecodeError:
    STUDY = pd.read_csv(fnm, encoding='latin1')
    # these work too
    # STUDY = pd.read_csv(fnm, encoding='ISO-8859-1')
    # STUDY = pd.read_csv(fnm, encoding='Windows-1252')


STUDY

,project_name,project_dataset,project_description,ASAP_team_name,ASAP_lab_name,PI_full_name,PI_email,contributor_names,submitter_name,submittor_email,...,other_funding_source,publication_DOI,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,metadata_version_date
0,Activation of transposable elements as a trigg...,Single nuclei sequencing of brain regions from...,"A range of genetic, clinical and pathological ...",TEAM-JAKOBSSON,Gale-Hammell Lab,"Molly, Gale Hammell",Molly.GaleHammell@nyulangone.org,"Anita, Adami; Talitha, Forcier; Raquel, Garza;...","Oliver, H, Tam",oliver.tam@nyulangone.org,...,NaN,NaN,NaN,71,"Substantia nigra, prefrontal cortex, amygdala,...",Late stage PD & control,NaN,NaN,Creative Commons Attribution license CC BY 4.0,NaN


Test reading the CDE from google sheets.

In [3]:



# # ASAP CDE ALL sheet
# sheet_id = "1SYyOw0zppqdRN-Jwlx6m8ZcBkqZgokjcO39bw6UwF8I"
# url_1a = "https://docs.google.com/spreadsheets/d/1SYyOw0zppqdRN-Jwlx6m8ZcBkqZgokjcO39bw6UwF8I/edit?usp=sharing"
# url_1b = "https://docs.google.com/spreadsheets/d/1SYyOw0zppqdRN-Jwlx6m8ZcBkqZgokjcO39bw6UwF8I/edit#gid=43504703"
# sheet_name = "CDE_final"
# url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"


# ASAP_CDE sheet
url_2a = "https://docs.google.com/spreadsheets/d/1xjxLftAyD0B8mPuOKUp5cKMKjkcsrp_zr9yuVULBLG8/edit#gid=1120395413"
url_2b = "https://docs.google.com/spreadsheets/d/1xjxLftAyD0B8mPuOKUp5cKMKjkcsrp_zr9yuVULBLG8/edit?usp=sharing"
sheet_id = "1xjxLftAyD0B8mPuOKUp5cKMKjkcsrp_zr9yuVULBLG8"
sheet_name = "ASAP_CDE_v2"  # sheet_name = "ASAP_CDE_v1"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
pd.read_csv(url).head()

,Table,Field,Description,DataType,Required,Validation
0,STUDY,ASAP_team_name,ASAP Team Name: Name of the ASAP CRN Team. i...,Enum,Required,"[""TEAM-LEE"",""TEAM-HAFLER"",""TEAM-HARDY"", ""TEAM-..."
1,STUDY,ASAP_lab_name,Lab Name. : Lab name that is submitting data...,String,Required,NaN
2,STUDY,project_name,Project Name: A Title of the overall project...,String,Required,NaN
3,STUDY,team_dataset_id,"The ""project_name"" is often too verbose for pr...",String,Required,NaN
4,STUDY,project_dataset,Dataset Name: A unique name is required for ...,String,Required,NaN


In [4]:

url_1 = url_2a.replace("/edit#gid=", "/export?format=csv&gid=")
pd.read_csv(url).head()                          

,Table,Field,Description,DataType,Required,Validation
0,STUDY,ASAP_team_name,ASAP Team Name: Name of the ASAP CRN Team. i...,Enum,Required,"[""TEAM-LEE"",""TEAM-HAFLER"",""TEAM-HARDY"", ""TEAM-..."
1,STUDY,ASAP_lab_name,Lab Name. : Lab name that is submitting data...,String,Required,NaN
2,STUDY,project_name,Project Name: A Title of the overall project...,String,Required,NaN
3,STUDY,team_dataset_id,"The ""project_name"" is often too verbose for pr...",String,Required,NaN
4,STUDY,project_dataset,Dataset Name: A unique name is required for ...,String,Required,NaN


## Clean each Team Table
write clean metadata tables according to CDE 

### Team Lee

In [20]:
## convert 
data_path = Path.home() / ("Projects/ASAP/team-lee")
metadata_path = data_path / "metadata/ogmetadata"

SUBJECT = pd.read_csv(f"{metadata_path}/SUBJECT.tsv", delimiter="\t")
SAMPLE = pd.read_csv(f"{metadata_path}/SAMPLE.tsv",delimiter="\t")

CLINPATH = pd.read_csv(f"{metadata_path}/CLINPATH.csv",delimiter=",")
STUDY = pd.read_csv(f"{metadata_path}/STUDY.tsv",delimiter="\t")
PROTOCOL = pd.read_csv(f"{metadata_path}/PROTOCOL.tsv",delimiter="\t")

metadata_version = "v1"

CDE_df,dtypes_dict = read_ASAP_CDE(metadata_version=metadata_version)
METADATA_VERSION_DATE = f"{metadata_version}_{pd.Timestamp.now().strftime('%Y%m%d')}"

read url


In [21]:
STUDY = pd.read_csv(metadata_path / "STUDY.tsv",delimiter="\t")
STUDY.to_csv(data_path / "STUDY_.csv")
STUDY = pd.read_csv(data_path / "STUDY_.csv")
STUDY.head()


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Team-Lee-Bras-Lab-Info,Field,Description,Data type,Validation,Note,Required/Optional
0,Is senescence a component of human PD and does...,project_name,Project Name/Title,String,NaN,Unique and clear title.,Required,NaN,NaN,NaN
1,Human snRNA-seq PD Senesence Jose Bras Team Lee,project_dataset,Dataset name,String,NaN,A Dataset name is required for each submission...,Required,NaN,NaN,NaN
2,Characterize the neuropathological progression...,project_description,Brief description of the goals and objectives ...,String,NaN,NaN,Required,NaN,NaN,NaN
3,TEAM-LEE,ASAP_team_name,"ASAP Team e.g. ""Scherzer""",Enum,"[""TEAM-LEE"",""TEAM-HAFLER"",""TEAM-HARDY"",....]",NaN,Required,NaN,NaN,NaN
4,Bras,ASAP_lab_name,"ASAP Lab under the above team e.g. ""Dong""",String,NaN,NaN,Required,NaN,NaN,NaN


In [22]:

# fix STUDY formatting
tmp = pd.DataFrame()
tmp = STUDY[["Unnamed: 1","Unnamed: 0"]].transpose().reset_index().drop(columns=["index"])
tmp.columns = tmp.iloc[0]
STUDY = tmp.drop([0])
# STUDY[["Unnamed: 1"]].transpose().reset_index().drop(columns=["index"]), tmp
STUDY.head()

,project_name,project_dataset,project_description,ASAP_team_name,ASAP_lab_name,PI_full_name,PI_email,submitter_id,submitter_name,submittor_email,...,other_funding_source,publication_DOI,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,metadata_version_date
1,Is senescence a component of human PD and does...,Human snRNA-seq PD Senesence Jose Bras Team Lee,Characterize the neuropathological progression...,TEAM-LEE,Bras,"Jose, Bras",jose.bras@vai.org,"Lee, L, Marshall ; Kimberly, E, Paquette ; Kai...",Kaitlyn E Westra,kaitlyn.westra@vai.org,...,NaN,NaN,NaN,75,hippocampus; middle frontal gyrus; substantia ...,human PD and control postmortem brains,NaN,NaN,unsure,NaN


In [24]:
metadata_version_date = "v1, SEPT2023"
STUDY["metadata_version_date"] = METADATA_VERSION_DATE
# Testing the function with STUDY.csv and CDE.csv
STUDY.replace("Nan", "", inplace=True)

# Need to rename submitter_id to contributor_names
STUDY = STUDY.rename(columns={"submitter_id": "contributor_names"})

report = ReportCollector(destination="")

validate_table(STUDY, "STUDY", CDE_df, report)

report.print_log()


All required fields are present in *STUDY* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in STUDY. 🥳



Collect additional metadata from covar.csv .. i.e. batch

In [25]:

metadata_path = Path.home() / ("Projects/ASAP/team-lee/metadata")
HIP_covar = pd.read_csv(f"{metadata_path}/HIP/covar.csv")
HIP_cases = pd.read_csv(f"{metadata_path}/HIP/PD_ASAP_Sample_batch_information_banner_cases.csv").dropna(axis=0,how='all')
HIP_control = pd.read_csv(f"{metadata_path}/HIP/PD_ASAP_Sample_batch_information_banner_controls.csv")

MFG_covar = pd.read_csv(f"{metadata_path}/MFG/covar.csv") # includes 'PMI' ?
MFG_cases = pd.read_csv(f"{metadata_path}/MFG/PD_ASAP_Sample_batch_information_banner_cases.csv").dropna(axis=0,how='all')
MFG_control = pd.read_csv(f"{metadata_path}/MFG/PD_ASAP_Sample_batch_information_banner_controls.csv")


SN_covar = pd.read_csv(f"{metadata_path}/SN/covar.csv")
SN_cases = pd.read_csv(f"{metadata_path}/SN/PD_ASAP_Sample_batch_information_banner_cases.csv").dropna(axis=0,how='all')
SN_control = pd.read_csv(f"{metadata_path}/SN/PD_ASAP_Sample_batch_information_banner_controls.csv")

In [26]:
# Hippocampus samples
# HIP_cases["GROUPcv"]="PD"
# HIP_control["GROUPcv"]="HC"

HIP_meta = pd.concat([HIP_cases, HIP_control], axis=0, ignore_index=True)
HIP_meta["GROUPcv"]= HIP_meta["PD"].apply(lambda x: "PD" if (x=="yes") else "HC")


In [27]:


HIP_meta['MERGE_ID'] = "HIP_" + HIP_meta['GROUPcv'] +"_" + HIP_meta['CaseID'].str.replace('-','')
HIP_covar['MERGE_ID'] = HIP_covar['COUNT_ID']
# the fastqs follow COUNT_ID insteald of SEQ_ID naming convention
HIP_covar['SEQ_ID'] = HIP_covar['COUNT_ID']



In [28]:
# there's a bug in the meta table... skip for now
HIP_TABLE = pd.merge(HIP_covar, HIP_meta, on='MERGE_ID', how='outer')

# HIP_TABLE = HIP_covar
HIP_TABLE['subdir']="HIP"


In [29]:
test = HIP_TABLE[["MERGE_ID","SEQ_ID","GROUPcv","subdir",'PD']]

In [30]:
### medial frontal gyrus samples
MFG_meta = pd.concat([MFG_cases, MFG_control], axis=0, ignore_index=True)
MFG_meta["GROUPcv"]= MFG_meta["PD"].apply(lambda x: "PD" if (x=="yes") else "HC")

# make a MERGE_ID column because the formatting is inconsistent
MFG_meta['MERGE_ID'] = "MFG_" + MFG_meta['GROUPcv'] +"_" + MFG_meta['CaseID'].str.replace('-','')
MFG_covar['MERGE_ID'] = MFG_covar['SAMPLE']
# the fastqs are in SEQ_ID 

# there's a bug in the meta table... skip for now
MFG_TABLE = pd.merge(MFG_covar, MFG_meta, on='MERGE_ID', how='inner')
MFG_TABLE['subdir']="MFG"



# Substantia Nigra
SN_meta = pd.concat([SN_cases, SN_control], axis=0, ignore_index=True)
SN_meta["GROUPcv"] = SN_meta["PD"].apply(lambda x: "PD" if (x=="yes") else "HC")

SN_meta['MERGE_ID'] = "SN_" + MFG_meta['GROUPcv'] +"_" + MFG_meta['CaseID'].str.replace('-','')
SN_covar['MERGE_ID'] = SN_covar['SAMPLE']

# there's a bug in the meta table... skip for now
SN_TABLE = pd.merge(SN_covar, SN_meta, on='MERGE_ID', how='outer')
SN_TABLE['subdir']="SN"


### concatenate SN, MSG, and HIP tables into one 'all_samples' table
all_samples = pd.concat([HIP_TABLE, MFG_TABLE, SN_TABLE], axis=0, ignore_index=True)


In [31]:

SAMPLE_ALL = SAMPLE.merge(all_samples, left_on='sample_id', right_on='MERGE_ID', how='left')
SAMPLE_ALL.to_csv("alternate_metadata.csv")

In [32]:
SAMPLE_og = SAMPLE.copy()
SAMPLE['batch'] = SAMPLE_ALL['BATCH']

In [34]:
# Extract the fields with DataType as "Enum" or "String" for the "sample" table from CDE.csv
SAMPLE = prep_table(SAMPLE, CDE_df)
# for field in string_enum_fields:
#     if field in SAMPLE.columns:
#         SAMPLE[field] = SAMPLE[field].astype(str)


In [35]:
# Testing the function with PROTOCOL.csv and CDE.csv

report = ReportCollector(destination="")
validate_table(PROTOCOL, "PROTOCOL", CDE, report)
report.print_log()

All required fields are present in *PROTOCOL* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in PROTOCOL. 🥳



In [36]:
# Extract the fields with DataType as "Enum" or "String" for the "sample" table from CDE.csv
SUBJECT.replace("Nan", "", inplace=True)


SUBJECT = prep_table(SUBJECT, CDE_df)


report = ReportCollector(destination="logs")
validate_table(SUBJECT, "SUBJECT", CDE, report)
report.print_log()

All required fields are present in *SUBJECT* table.
No empty entries (Nan) found in _Required_ fields.
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- primary_diagnosis_text: 23/25 empty rows
## Enum fields have valid values in SUBJECT. 🥳



In [37]:
SAMPLE.replace("Nan", "", inplace=True)

SAMPLE = prep_table(SAMPLE, CDE_df)


In [38]:
SAMPLE['file_type'] = SAMPLE['file_type'].replace({"Fastq":"fastq"})


In [39]:
# fix file_name and file_MD5 which need to be exploded (do this last for simplicity. i.e. to keep one sample per row rather than one file per row)

# Step 1: Split the values in the columns based on commas
SAMPLE['file_name'] = SAMPLE['file_name'].str.split(',')
SAMPLE['file_MD5(R1,R2)'] = SAMPLE['file_MD5(R1,R2)'].str.split(',')

# Step 2: Explode both 'file_name' and 'file_MD5(R1,R2)' columns together
SAMPLE = SAMPLE.explode(['file_name', 'file_MD5(R1,R2)'])

# Step 3: Rename the "file_MD5(R1,R2)" column to "file_MD5"
SAMPLE = SAMPLE.rename(columns={"file_MD5(R1,R2)": "file_MD5"})



In [41]:


report = ReportCollector(destination="")
validate_table(SAMPLE, "SAMPLE", CDE_df, report)
report.print_log()


All required fields are present in *SAMPLE* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- source_RIN: 150/150 empty rows

	- RIN: 150/150 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- pm_PH: 150/150 empty rows
## Enum fields have valid values in SAMPLE. 🥳



In [30]:
SAMPLE.head()

,sample_id,source_sample_id,subject_id,replicate,replicate_count,repeated_sample,tissue,brain_region,source_RIN,RIN,...,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV2000,pm_PH,donor_id,batch
0,MFG_HC_1225,12-25,12-25,Rep1,1,0,Brain,Middle_Frontal_Gyrus,NaN,NaN,...,Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NaN,NaN,,BATCH_4
0,MFG_HC_1225,12-25,12-25,Rep1,1,0,Brain,Middle_Frontal_Gyrus,NaN,NaN,...,Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NaN,NaN,,BATCH_4
1,MFG_HC_0602,06-02,06-02,Rep1,1,0,Brain,Middle_Frontal_Gyrus,NaN,NaN,...,Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NaN,NaN,,BATCH_4
1,MFG_HC_0602,06-02,06-02,Rep1,1,0,Brain,Middle_Frontal_Gyrus,NaN,NaN,...,Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NaN,NaN,,BATCH_4
2,MFG_PD_0009,00-09,00-09,Rep1,1,0,Brain,Middle_Frontal_Gyrus,NaN,NaN,...,Unknown,MONDO:0005180,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NaN,NaN,,BATCH_4


In [42]:
CLINPATH = prep_table(CLINPATH, CDE_df)


CLINPATH.replace("Nan", "", inplace=True)

report = ReportCollector(destination="")
validate_table(CLINPATH, "CLINPATH", CDE_df, report)
report.print_log()


All required fields are present in *CLINPATH* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- age_at_onset: 75/75 empty rows

	- age_at_diagnosis: 75/75 empty rows

	- first_motor_symptom: 75/75 empty rows

	- path_year_death: 75/75 empty rows

	- brain_weight: 75/75 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- smoking_years: 75/75 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*region_level_2*_:  invalid values 💩'Hippocampus'
    - valid ➡️ 'Superior frontal gyrus', 'Middle frontal gyrus', 'Inferior frontal gyrus', 'Superior temporal gyrus', 'Middle temporal gyrus', 'Inferior temporal gyrus', 'Fusiform gyrus', 'Transentorhinal region', 'Entorinal region', 'Subiculum', 'CA1-CA4', 'Amygdala', 'Periamygdala cortex', 'Anterior cingulate gyrus', 'Posterior cingulate gyrus', 'Superior parietal lobule', 'Inferior parietal lobule', 'Parastriate cortex', 'Peristriate cortex', 'Striate cortex', 'Insular cortex', 'Caudate nucleus', 'Putamen', 'Globus pallidus', 

In [43]:
CLINPATH['region_level_2'].unique()

# change "Hippocampus" to "CA1-CA4"
CLINPATH['region_level_2'] = CLINPATH['region_level_2'].replace('Hippocampus', 'CA1-CA4')

# skip hx_melanoma and education level for now as there is not a "Unknown" or "Not Reported" option in the CDE

# leave te APOE_e4_status as is for now . multiple are coded as "2,3" 
# leave cognitive status as is, since there is no "Unknown" or "Not Reported" option in the CDE

# potential "path_braak_asyn" coding 
braak_map = {'L. Olfactory Bulb-Only':"1/2", 'Lla. Brainstem Predominant':"3",
       'Llb. Limbic Predominant':"3/4", 'LV. Neocortical':"5",
       'Lll. Brainstem/Limbic':"3/4", '0. No Lewy bodies':"0"}
# set to NaN for now since this is actualy path_mckeith coding

CLINPATH['path_braak_asyn'] = ""

mckeith_map = {'L. Olfactory Bulb-Only':"Olfactory bulb only", 'Lla. Brainstem Predominant':"Brainstem",
       'Llb. Limbic Predominant':"Limbic (transitional)", 'LV. Neocortical':"Neocortical",
       'Lll. Brainstem/Limbic':"Amygdala Predominant", '0. No Lewy bodies':"Absent"}


CLINPATH['path_mckeith'] = CLINPATH['path_mckeith'].replace(mckeith_map)

# leave path_nia_ri like this for now. not sure how to map "criteria not met" and "Not AD"

# leave amyloid_angiopathy_severity_scale like this for now. not sure how to map 'Cerebral amyloid angiopathy, temporal and occipital lobe','Cerebral amyloid angiopathy, frontal lobe']


In [215]:
validate_table(CLINPATH, "CLINPATH", CDE)

	All required fields are present in CLINPATH.
		Required Fields with Empty or NaN values:
			- age_at_onset: 75 rows
			- age_at_diagnosis: 75 rows
			- first_motor_symptom: 75 rows
			- path_year_death: 75 rows
			- brain_weight: 75 rows
		Optional Fields with Empty or NaN values:
			- smoking_years: 75 rows
	Invalid Field/Value pairs:
			- path_nia_ri: Criteria not met, Not AD
			- TDP43: , Na
			- amyloid_angiopathy_severity_scale: , Cerebral amyloid angiopathy, temporal and occipital lobe, Cerebral amyloid angiopathy, frontal lobe
			- path_ad_level: Microscopic changes of Alzheimer's disease, insufficient for diagnosis, Microscopic lesions of Alzheimer's disease, insufficient for diagnosis


0

In [44]:
report = ReportCollector(destination="")
validate_table(CLINPATH, "CLINPATH", CDE_df, report)
report.print_log()


All required fields are present in *CLINPATH* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- age_at_onset: 75/75 empty rows

	- age_at_diagnosis: 75/75 empty rows

	- first_motor_symptom: 75/75 empty rows

	- path_year_death: 75/75 empty rows

	- brain_weight: 75/75 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- smoking_years: 75/75 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*path_nia_ri*_:  invalid values 💩'Criteria not met', 'Not AD'
    - valid ➡️ 'Low', 'Intermediate', 'High', 'None'
- _*TDP43*_:  invalid values 💩'', 'Na'
    - valid ➡️ 'None in medial temporal lobe', 'Present in amygdala, only', 'Present in hippocampus, only', 'Present in amygdala and hippocampus, only', 'Present in medial temporal lobe and middle frontal gyrus (not FTLD pattern)', 'Unknown'
- _*amyloid_angiopathy_severity_scale*_:  invalid values 💩'', 'Cerebral amyloid angiopathy, temporal and occipital lobe', 'Cerebral amyloid angiopathy, frontal lobe'
    - valid ➡️ 'None', 

In [45]:

SAMPLE_ALL_CP = SAMPLE_ALL.merge(CLINPATH, on='sample_id', how='outer')


In [46]:
SAMPLE_ALL_CP.to_csv("./clean/team-Lee/auxiluary_metadata.csv")

In [48]:
# fix the column order
STUDY = reorder_table_to_CDE(STUDY, "STUDY", CDE)
SAMPLE = reorder_table_to_CDE(SAMPLE, "SAMPLE", CDE)
PROTOCOL = reorder_table_to_CDE(PROTOCOL, "PROTOCOL", CDE)
SUBJECT = reorder_table_to_CDE(SUBJECT, "SUBJECT", CDE)     
CLINPATH = reorder_table_to_CDE(CLINPATH, "CLINPATH", CDE)

# write the clean metadata
STUDY.to_csv(data_path / "metadata/STUDY.csv")
PROTOCOL.to_csv(data_path / "metadata/PROTOCOL.csv")
CLINPATH.to_csv(data_path / "metadata/CLINPATH.csv")
SAMPLE.to_csv(data_path / "metadata/SAMPLE.csv")
SUBJECT.to_csv(data_path / "metadata/SUBJECT.csv")

# also writh them to clean...
# 
#  

export_root = Path.cwd() / "clean/team-Lee"
if not export_root.exists():
    export_root.mkdir(parents=True, exist_ok=True)


STUDY.to_csv( export_root / "STUDY.csv")
PROTOCOL.to_csv(export_root / "PROTOCOL.csv")
SAMPLE.to_csv(export_root / "SAMPLE.csv")
SUBJECT.to_csv(export_root / "SUBJECT.csv")
CLINPATH.to_csv(export_root / "CLINPATH.csv")


## Team Hafler

In [78]:
## convert to seurat Object
data_path = Path.home() / ("Projects/ASAP")
metadata_path = data_path / "team-hafler/metadata"

sheets = ["SAMPLE","SUBJECT","CLINPATH","STUDY","PROTOCOL"]
excel_path = metadata_path / "ASAP_CDE_ALL_Team_Hafler_v1.xlsx"
STUDY = pd.read_excel(excel_path,sheet_name="STUDY",header=1).drop(columns="Field")
CLINPATH = pd.read_excel(excel_path,sheet_name="CLINPATH",header=1).drop(columns="Field")
SUBJECT = pd.read_excel(excel_path,sheet_name="SUBJECT",header=1).drop(columns="Field")
SAMPLE = pd.read_excel(excel_path,sheet_name="SAMPLE",header=1).drop(columns="Field")
PROTOCOL = pd.read_excel(excel_path,sheet_name="PROTOCOL",header=1).drop(columns="Field")
metadata_path

PosixPath('/Users/ergonyc/Projects/ASAP/team-hafler/metadata')

In [50]:

def add_hafler_batch(sample_df):

    # First batch: HSDG07HC HSDG10HC HSDG148PD HSDG199PD
    # batch[batch.sample_id in ['hSDG07HC', 'hSDG10HC', 'hSDG148PD', 'hSDG199PD']]=1
    Batch_1 = ['hSDG07', 'hSDG10', 'hSDG148', 'hSDG199'] 
    # Second batch: hsDG101HC hsDG13HC hsDG151PD hsDG197PD hsDG30HC hsDG99HC
    Batch_2 = ['hSDG101', 'hSDG13', 'hSDG151', 'hSDG197', 'hSDG30', 'hSDG99']
    # Third batch: hsDG142PD hsDG208PD
    Batch_3 = ['hSDG142', 'hSDG208'] 


    batch_col = []
    for row in sample_df.sample_id:
        if row in Batch_1:
            batch_col.append("Batch_1")
        elif row in Batch_2:
            batch_col.append("Batch_2")
        elif row in Batch_3:
            batch_col.append("Batch_3")
        else:
            print("ERROR >>>>>>>> not no batch info")
            batch_col.append("")


    sample_df['batch'] = batch_col
    return sample_df

SAMPLE = add_hafler_batch(SAMPLE)

In [52]:
# fix replicate & replicate_count
SAMPLE.replace("Nan", "", inplace=True)

SAMPLE['replicate'] = "Rep1"
SAMPLE['replicate_count'] = 1

SAMPLE.head()

,sample_id,source_sample_id,subject_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,source_RIN,...,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV2000,pm_PH,donor_id
0,hSDG07,hSDG07,HC01,Rep1,1,0,Batch_1,Brain,Prefrontal Cortex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,hSDG07,hSDG07,HC01,Rep1,1,0,Batch_1,Brain,Prefrontal Cortex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,hSDG07,hSDG07,HC01,Rep1,1,0,Batch_1,Brain,Prefrontal Cortex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,hSDG101,hSDG101,HC03,Rep1,1,0,Batch_2,Brain,Prefrontal Cortex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,hSDG101,hSDG101,HC03,Rep1,1,0,Batch_2,Brain,Prefrontal Cortex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:

STUDY = prep_table(STUDY, CDE_df)
STUDY['metadata_version_date']

0    V1_08/27/2023
Name: metadata_version_date, dtype: object

In [55]:
# update metadata_version_date
STUDY['metadata_version_date'] = METADATA_VERSION_DATE

In [225]:
report = ReportCollector(destination="")
validate_table(STUDY, "STUDY", CDE_df, report)
report.print_log()


	All required fields are present in STUDY.
No empty or NaN values found in Required fields.
No empty or NaN values found in Optional fields.
	All Enum fields have valid values in STUDY.


1

In [56]:
report = ReportCollector(destination="")
validate_table(PROTOCOL, "PROTOCOL", CDE_df, report)
report.print_log()


All required fields are present in *PROTOCOL* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in PROTOCOL. 🥳



In [57]:
SUBJECT.replace("Nan", "", inplace=True)
# Testing the function with SUBJECT.csv and CDE.csv
report = ReportCollector(destination="")
validate_table(SUBJECT, "SUBJECT", CDE_df, report)
report.print_log()


All required fields are present in *SUBJECT* table.
No empty entries (Nan) found in _Required_ fields.
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- primary_diagnosis_text: 12/12 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*sex*_:  invalid values 💩'F', 'M'
    - valid ➡️ 'Male', 'Female', 'Intersex', 'Unnown'
- _*race*_:  invalid values 💩'B', 'W'
    - valid ➡️ 'American Indian or Alaska Native', 'Asian', 'White', 'Black or African American', 'Multi-racial', 'Native Hawaiian or Other Pacific Islander', 'Other', 'Unknown', 'Not Reported'
- _*ethnicity*_:  invalid values 💩'B', 'W'
    - valid ➡️ 'Hispanic or Latino', 'Not Hispanic or Latino', 'Unknown', 'Not Reported'
- _*primary_diagnosis*_:  invalid values 💩'Normal control', 'Idiopathic Parkinson's disease'
    - valid ➡️ 'Healthy Control', 'Idiopathic PD', 'Alzheimer’s disease', 'Frontotemporal dementia', 'Corticobasal syndrome', 'Dementia with Lewy bodies', 'Dopa-responsive dystonia', 'Essential tremor', 'Hemiparkinson/he

In [58]:

SUBJECT = prep_table(SUBJECT, CDE_df)


In [59]:
SUBJECT['sex'] = SUBJECT['sex'].replace({'F':"Female", 'M':"Male"})
SUBJECT['race'] = SUBJECT['race'].replace({'W':"White", 'B':"Black or African American"})

SUBJECT['primary_diagnosis'] = SUBJECT['primary_diagnosis'].replace({'Normal control':"Healthy Control", "Idiopathic Parkinson's disease":"Idiopathic PD"})
report = ReportCollector(destination="")
validate_table(SUBJECT, "SUBJECT", CDE_df, report)
report.print_log()


All required fields are present in *SUBJECT* table.
No empty entries (Nan) found in _Required_ fields.
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- primary_diagnosis_text: 12/12 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*ethnicity*_:  invalid values 💩'B', 'W'
    - valid ➡️ 'Hispanic or Latino', 'Not Hispanic or Latino', 'Unknown', 'Not Reported'



In [63]:
# they "race" was reported under ethnicity, so code as "Not Reported"
SUBJECT['ethnicity'] = 'Not Reported'

In [64]:

report = ReportCollector(destination="")
validate_table(SUBJECT, "SUBJECT", CDE_df, report)
report.print_log()


All required fields are present in *SUBJECT* table.
No empty entries (Nan) found in _Required_ fields.
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- primary_diagnosis_text: 12/12 empty rows
## Enum fields have valid values in SUBJECT. 🥳



In [65]:
SAMPLE = prep_table(SAMPLE,  CDE_df)


In [66]:
# force the right sex_ontology_term_id
SAMPLE["organism_ontology_term_id"] = "NCBITaxon:9606"

# set time == 0 for all samples
SAMPLE['time'] = 0

SAMPLE['file_type'] = SAMPLE['file_type'].replace({"Fastq":"fastq"})


In [67]:

# need to join with subject to get "sex" and convert to ontology term
SAMPLE_SUBJECT = SAMPLE.merge(SUBJECT, on='subject_id',  how='left')
SAMPLE_og = SAMPLE.copy()
SAMPLE['sex_ontology_term_id'] = SAMPLE_SUBJECT['sex'].replace({"Male":"PATO:0000384 (male)", "Female":"PATO:0000383 (female)" })

# ignore development_stage_ontology_term_id, self_reported_ethnicity_ontology_term_id, assay_ontology_term_id, etc for now. (Check wiht Le)

In [68]:
# fix assay
SAMPLE['assay'] = SAMPLE['assay'].replace({'v3.1 - Single Index, 10x Genomics ':"v3.1 - Single Index"})
# fix assay
SAMPLE['sequencing_length'] = SAMPLE['sequencing_length'].replace({'150bp x2':"150"})


In [69]:

report = ReportCollector(destination="")
validate_table(SAMPLE, "SAMPLE", CDE_df, report)
report.print_log()


All required fields are present in *SAMPLE* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- source_RIN: 36/36 empty rows

	- RIN: 36/36 empty rows

	- suspension_type: 36/36 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- pm_PH: 36/36 empty rows
## Enum fields have valid values in SAMPLE. 🥳



In [70]:
# CLINPATH.replace("Nan", "", inplace=True)
# CLINPATH.replace("nan", "", inplace=True)
report = ReportCollector(destination="")
validate_table(CLINPATH, "CLINPATH", CDE_df, report)
report.print_log()


🚨⚠️❗ **Missing Required Fields in CLINPATH: path_thal**
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- age_at_onset: 12/12 empty rows

	- age_at_diagnosis: 12/12 empty rows

	- first_motor_symptom: 12/12 empty rows

	- path_year_death: 12/12 empty rows

	- brain_weight: 12/12 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- smoking_years: 12/12 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*region_level_2*_:  invalid values 💩'Prefrontal cortex'
    - valid ➡️ 'Superior frontal gyrus', 'Middle frontal gyrus', 'Inferior frontal gyrus', 'Superior temporal gyrus', 'Middle temporal gyrus', 'Inferior temporal gyrus', 'Fusiform gyrus', 'Transentorhinal region', 'Entorinal region', 'Subiculum', 'CA1-CA4', 'Amygdala', 'Periamygdala cortex', 'Anterior cingulate gyrus', 'Posterior cingulate gyrus', 'Superior parietal lobule', 'Inferior parietal lobule', 'Parastriate cortex', 'Peristriate cortex', 'Striate cortex', 'Insular cortex', 'Caudate nucleus', 'Putamen', 'Globus pa

In [71]:
CLINPATH = prep_table(CLINPATH, CDE_df)


In [72]:
# redact "Prefrontal Cortex" from region_level_2 for now
CLINPATH['region_level_2'] = CLINPATH['region_level_2'].replace({'Prefrontal Cortex':""})

# leave te APOE_e4_status as is for now . multiple are coded as "2,3" 
# but remove commas
CLINPATH["APOE_e4_status"] = CLINPATH["APOE_e4_status"].str.replace(",","")

# need to fix the path_autopsy_dx_main

In [73]:
report = ReportCollector(destination="")
validate_table(CLINPATH, "CLINPATH", CDE_df, report)
report.print_log()

🚨⚠️❗ **Missing Required Fields in CLINPATH: path_thal**
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- age_at_onset: 12/12 empty rows

	- age_at_diagnosis: 12/12 empty rows

	- first_motor_symptom: 12/12 empty rows

	- path_year_death: 12/12 empty rows

	- brain_weight: 12/12 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- smoking_years: 12/12 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*region_level_2*_:  invalid values 💩'Prefrontal cortex'
    - valid ➡️ 'Superior frontal gyrus', 'Middle frontal gyrus', 'Inferior frontal gyrus', 'Superior temporal gyrus', 'Middle temporal gyrus', 'Inferior temporal gyrus', 'Fusiform gyrus', 'Transentorhinal region', 'Entorinal region', 'Subiculum', 'CA1-CA4', 'Amygdala', 'Periamygdala cortex', 'Anterior cingulate gyrus', 'Posterior cingulate gyrus', 'Superior parietal lobule', 'Inferior parietal lobule', 'Parastriate cortex', 'Peristriate cortex', 'Striate cortex', 'Insular cortex', 'Caudate nucleus', 'Putamen', 'Globus pa

In [74]:
data_path = data_path / "team-hafler"
data_path = Path.home() / ("Projects/ASAP/team-hafler")

In [75]:
# fix the column order
STUDY = reorder_table_to_CDE(STUDY, "STUDY", CDE)
SAMPLE = reorder_table_to_CDE(SAMPLE, "SAMPLE", CDE)
PROTOCOL = reorder_table_to_CDE(PROTOCOL, "PROTOCOL", CDE)
SUBJECT = reorder_table_to_CDE(SUBJECT, "SUBJECT", CDE)     
CLINPATH = reorder_table_to_CDE(CLINPATH, "CLINPATH", CDE)

# write the clean metadata
STUDY.to_csv(data_path / "metadata/STUDY.csv")
PROTOCOL.to_csv(data_path / "metadata/PROTOCOL.csv")
CLINPATH.to_csv(data_path / "metadata/CLINPATH.csv")
SAMPLE.to_csv(data_path / "metadata/SAMPLE.csv")
SUBJECT.to_csv(data_path / "metadata/SUBJECT.csv")

# also writh them to clean...
# 
#  

export_root = Path.cwd() / "clean/team-Hafler"
if not export_root.exists():
    export_root.mkdir(parents=True, exist_ok=True)


STUDY.to_csv( export_root / "STUDY.csv")
PROTOCOL.to_csv(export_root / "PROTOCOL.csv")
SAMPLE.to_csv(export_root / "SAMPLE.csv")
SUBJECT.to_csv(export_root / "SUBJECT.csv")
CLINPATH.to_csv(export_root / "CLINPATH.csv")


In [76]:
# STUDY.to_csv( metadata_path / "STUDY.csv")
# CLINPATH.to_csv( metadata_path / "CLINPATH.csv")
# SUBJECT.to_csv( metadata_path / "SUBJECT.csv")
# SAMPLE.to_csv( metadata_path / "SAMPLE.csv")
# PROTOCOL.to_csv( metadata_path / "PROTOCOL.csv")

In [77]:
CLINPATH["APOE_e4_status"].unique()

CDE[CDE["Field"]== "APOE_e4_status"].Validation.unique()

array(['["22", "23","24","33", "34", "44", "Unknown"]'], dtype=object)

## Team Hardy

In [5]:
## convert 
data_path = Path.home() / ("Projects/ASAP/team-hardy")
metadata_path = data_path / "metadata"

SUBJECT = pd.read_csv(f"{metadata_path}/SUBJECT.csv")
CLINPATH = pd.read_csv(f"{metadata_path}/CLINPATH.csv")
STUDY = pd.read_csv(f"{metadata_path}/STUDY.csv")
PROTOCOL = pd.read_csv(f"{metadata_path}/PROTOCOL.csv")
SAMPLE = pd.read_csv(f"{metadata_path}/SAMPLE.csv")

metadata_version = "v1"

CDE_df,dtypes_dict = read_ASAP_CDE(metadata_version=metadata_version)
METADATA_VERSION_DATE = f"{metadata_version}_{pd.Timestamp.now().strftime('%Y%m%d')}"

read url


In [6]:

# fix STUDY formatting
tmp = pd.DataFrame()
tmp = STUDY[["name","value"]].transpose().reset_index().drop(columns=["index"])
tmp

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,project_name,project_dataset,project_description,ASAP_team_name,ASAP_lab_name,PI_full_name,PI_email,contributor_names,submitter_name,submitter_email,...,other_funding_source,publication_DOI,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,metadata_version_date
1,Understanding mechanisms of Parkinson's diseas...,Hardy snRNA-seq,Genetic analysis has identified many risk gene...,TEAM-HARDY,Ryten Lab,Mina Ryten,mina.ryten@ucl.ac.uk,"Aine Fairbrother-Browne, Jonathan Brenton, Mel...",Aine Fairbrother-Browne,aine.fairbrother-browne.18@ucl.ac.uk,...,NaN,NaN,NaN,128,"Inferior Parietal Lobule (IPL), Anterior Cingu...",Late stage (Braak 5-6) PD and control post-mor...,0000-0001-9520-6957,https://scholar.google.co.uk/citations?user=lt...,NaN,"Version 1, 09/2023"


In [7]:

tmp.columns = tmp.iloc[0]
STUDY = tmp.drop([0])
STUDY.head()


,project_name,project_dataset,project_description,ASAP_team_name,ASAP_lab_name,PI_full_name,PI_email,contributor_names,submitter_name,submitter_email,...,other_funding_source,publication_DOI,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,metadata_version_date
1,Understanding mechanisms of Parkinson's diseas...,Hardy snRNA-seq,Genetic analysis has identified many risk gene...,TEAM-HARDY,Ryten Lab,Mina Ryten,mina.ryten@ucl.ac.uk,"Aine Fairbrother-Browne, Jonathan Brenton, Mel...",Aine Fairbrother-Browne,aine.fairbrother-browne.18@ucl.ac.uk,...,NaN,NaN,NaN,128,"Inferior Parietal Lobule (IPL), Anterior Cingu...",Late stage (Braak 5-6) PD and control post-mor...,0000-0001-9520-6957,https://scholar.google.co.uk/citations?user=lt...,NaN,"Version 1, 09/2023"


In [9]:

STUDY = prep_table(STUDY, CDE_df)
# update metadata_version_date
STUDY['metadata_version_date'] = METADATA_VERSION_DATE


In [10]:
report = ReportCollector(destination="")
validate_table(STUDY, "STUDY", CDE_df, report)
report.print_log()


All required fields are present in *STUDY* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in STUDY. 🥳



In [11]:
STUDY.head()

,project_name,project_dataset,project_description,ASAP_team_name,ASAP_lab_name,PI_full_name,PI_email,contributor_names,submitter_name,submitter_email,...,other_funding_source,publication_DOI,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,metadata_version_date
1,Understanding mechanisms of Parkinson's diseas...,Hardy snRNA-seq,Genetic analysis has identified many risk gene...,TEAM-HARDY,Ryten Lab,Mina Ryten,Mina.ryten@ucl.ac.uk,"Aine Fairbrother-Browne, Jonathan Brenton, Mel...",Aine Fairbrother-Browne,Aine.fairbrother-browne.18@ucl.ac.uk,...,Nan,Nan,Nan,128,"Inferior Parietal Lobule (IPL), Anterior Cingu...",Late stage (Braak 5-6) PD and control post-mor...,0000-0001-9520-6957,Https://scholar.google.co.uk/citations?user=lt...,Nan,v1_20231114


In [12]:
# fix STUDY formatting
tmp = pd.DataFrame()
tmp = PROTOCOL[["name","value"]].transpose().reset_index().drop(columns=["index"])
tmp.columns = tmp.iloc[0]
PROTOCOL = tmp.drop([0])
PROTOCOL.head()

,sample_collection_summary,cell_extraction_summary,lib_prep_summary,data_processing_summary,github_url,protocols_io_DOI,other_reference
1,"This dataset contains cortical regions only, p...",From protocols.io: This protocol is used to is...,'Nuclei were extracted from homogenised post-m...,Cell ranger was used to convert raw sequencing...,Raw to fastq to mapped: https://github.com/RHR...,Nuclear extraction protocol: 10.17504/protocol...,NaN


In [13]:
# fix the column order
PROTOCOL = prep_table(PROTOCOL, CDE_df)
report = ReportCollector(destination="")
validate_table(PROTOCOL, "PROTOCOL", CDE_df, report)
report.print_log()



All required fields are present in *PROTOCOL* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in PROTOCOL. 🥳



In [14]:

SUBJECT = prep_table(SUBJECT, CDE_df)

report = ReportCollector(destination="")
validate_table(SUBJECT, "SUBJECT", CDE_df, report)
report.print_log()


All required fields are present in *SUBJECT* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- duration_pmi: 1/64 empty rows
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in SUBJECT. 🥳



In [15]:
SUBJECT.race.unique()[0]

'Nan'

In [16]:
SUBJECT = reorder_table_to_CDE(SUBJECT, "SUBJECT", CDE_df)
SUBJECT.head()

,subject_id,source_subject_id,biobank_name,organism,sex,age_at_collection,race,ethnicity,duration_pmi,primary_diagnosis,primary_diagnosis_text
0,Babom,P2/14,QSBB_UK,Human,Female,78,Nan,Nan,46.0,Idiopathic PD,clinpath info: PDD | PD (with dementia)
1,Borah,P4/11,QSBB_UK,Human,Male,63,Nan,Nan,37.0,Idiopathic PD,clinpath info: PD | PD
2,Bovon,P95/10,QSBB_UK,Human,Male,81,Nan,Nan,59.5,Idiopathic PD,clinpath info: PD | NA
3,Davof,P80/11,QSBB_UK,Human,Male,80,Nan,Nan,100.0,Idiopathic PD,clinpath info: PD | PD
4,Dudug,P82/10,QSBB_UK,Human,Female,87,Nan,Nan,84.0,No PD nor other neurological disorder,clinpath info: Control | Control


In [17]:
# def clean_empty_cols(df):
#     for col in df.columns:
#         if df[col].dtype in ["object", "string"]:
#             df[col].replace("Nan", pd.NA, inplace=True)
#             df[col].replace("nan", pd.NA, inplace=True)
#             df[col].replace("NA", pd.NA, inplace=True)

#         elif df[col].dtype in ["int", "float"]:
#             df[col].fillna(pd.NA, inplace=True)
#     return df


# SUBJECT = clean_empty_cols(SUBJECT)

SUBJECT.replace("Nan", pd.NA, inplace=True)
# SUBJECT.replace("nan", "", inplace=True)
# SUBJECT.fillna("", inplace=True)
SUBJECT.head()

,subject_id,source_subject_id,biobank_name,organism,sex,age_at_collection,race,ethnicity,duration_pmi,primary_diagnosis,primary_diagnosis_text
0,Babom,P2/14,QSBB_UK,Human,Female,78,<NA>,<NA>,46.0,Idiopathic PD,clinpath info: PDD | PD (with dementia)
1,Borah,P4/11,QSBB_UK,Human,Male,63,<NA>,<NA>,37.0,Idiopathic PD,clinpath info: PD | PD
2,Bovon,P95/10,QSBB_UK,Human,Male,81,<NA>,<NA>,59.5,Idiopathic PD,clinpath info: PD | NA
3,Davof,P80/11,QSBB_UK,Human,Male,80,<NA>,<NA>,100.0,Idiopathic PD,clinpath info: PD | PD
4,Dudug,P82/10,QSBB_UK,Human,Female,87,<NA>,<NA>,84.0,No PD nor other neurological disorder,clinpath info: Control | Control


In [18]:
SAMPLE = pd.read_csv(f"{metadata_path}/SAMPLE.csv")

# force the right sex_ontology_term_id
SAMPLE["organism_ontology_term_id"] = "NCBITaxon:9606"

SAMPLE = prep_table(SAMPLE, CDE_df)

# allow sequence_length == 190 for now

# SAMPLE.fillna("", inplace=True)
# remove any text nans
SAMPLE.replace("Nan", pd.NA, inplace=True)
SAMPLE.replace("nan", pd.NA, inplace=True)
SAMPLE.replace("NA", pd.NA, inplace=True)
# SAMPLE = clean_empty_cols(SUBJECT)
# SAMPLE.fillna("", inplace=True)

report = ReportCollector(destination="")
validate_table(SAMPLE, "SAMPLE", CDE_df, report)
report.print_log()




# add 'replicate' coding (nans)

🚨⚠️❗ **Missing Required Fields in SAMPLE: source_sample_id**
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- source_RIN: 3616/3616 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- pm_PH: 3616/3616 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*sequencing_length*_:  invalid values 💩'190'
    - valid ➡️ '25', '50', '100', '150'



In [34]:

SAMPLE['source_sample_id'] = pd.NA
SAMPLE['DV200'] = pd.NA
# SAMPLE = reorder_table_to_CDE(SAMPLE, "SAMPLE", CDE_df)


In [20]:
CLINPATH = pd.read_csv(f"{metadata_path}/CLINPATH.csv")

CLINPATH = prep_table(CLINPATH, CDE_df)


# CLINPATH.replace("Nan", pd.NA, inplace=True)
CLINPATH.replace("nan", "Nan", inplace=True)
CLINPATH.replace("NA", "Nan", inplace=True)

# CLINPATH.fillna("Nan")
report = ReportCollector(destination="")
validate_table(CLINPATH, "CLINPATH", CDE_df, report)
report.print_log()




🚨⚠️❗ **Missing Required Fields in CLINPATH: source_sample_id**
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- age_at_onset: 138/138 empty rows

	- age_at_diagnosis: 10/138 empty rows

	- first_motor_symptom: 138/138 empty rows

	- path_year_death: 138/138 empty rows

	- brain_weight: 138/138 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- smoking_years: 138/138 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*path_autopsy_dx_main*_:  invalid values 💩'Control brain', 'Pathological ageing', 'Control brain / Path ageing', 'Argyrophilic grain disease', 'Control brain, Cerebrovascular disease (small vessel)', 'Cerebrovascular disease (small vessel)', 'Control brain, Alzheimer`s disease (intermediate level AD pathological change)', 'Control brain / Path ageing, CAA'
    - valid ➡️ 'Lewy body disease nos', 'Parkinson's disease', 'Parkinson's disease with dementia', 'Dementia with Lewy bodies', 'Multiple system atrophy (SND>OPCA)', 'Multiple system atrophy (OPCA<SND)', 

In [21]:
na_float2intstr = lambda val: str(int(float(val))) if val != 'Nan' else pd.NA 

# replace 'path_braak_asyn' with with string of the numeric. converte nan to ""
CLINPATH['path_braak_asyn'] = CLINPATH['path_braak_asyn'].apply(na_float2intstr)



In [22]:

# replace 'path_braak_nft' with with string of the numeric. converte nan to ""
CLINPATH['path_braak_nft'] = CLINPATH['path_braak_nft'].apply(na_float2intstr).replace({"0":"0", 
                                                                                        "1":"I", 
                                                                                        "2": "II", 
                                                                                        "3":"III", 
                                                                                        "4":"IV", 
                                                                                        "5":"V", 
                                                                                        "6":"VI"})


In [23]:

# code family_history as "Not Reported" (currently empty)
CLINPATH['family_history'] = "Not Reported"



# check APOE_e4_status ? currently empty

# `path_autopsy_dx_main`  actually seems good parser might be wrong

# code "at least 4" as "4/5" 

CLINPATH['path_thal'] = CLINPATH['path_thal'].replace({'At least 4':"4/5"})


CLINPATH['path_mckeith'] = CLINPATH['path_mckeith'].replace({'Diffuse neocortical': "Diffuse, neocortical (brainstem, limbic and neocortical involvement)", 
                                                        'Limbic transitional': "Limbic (transitional)" ,
                                                        'Diffuse Neocortical':"Diffuse, neocortical (brainstem, limbic and neocortical involvement)"})


In [24]:

CLINPATH['path_nia_aa_a'] = CLINPATH['path_nia_aa_a'].apply(na_float2intstr).replace({"0":"A0", 
                                                                                    "1":"A1", 
                                                                                    "2": "A2", 
                                                                                    "3":"A3"})


In [25]:

# replace 'path_braak_nft' with with string of the numeric. converte nan to ""
CLINPATH['path_nia_aa_b'] = CLINPATH['path_nia_aa_b'].apply(na_float2intstr).replace({"0":"B0", 
                                                                                    "1":"B1", 
                                                                                    "2": "B2", 
                                                                                    "3":"B3"})


# replace 'path_braak_nft' with with string of the numeric. converte nan to ""
CLINPATH['path_nia_aa_c'] = CLINPATH['path_nia_aa_c'].apply(na_float2intstr).replace({"0":"C0", 
                                                                                    "1":"C1", 
                                                                                    "2": "C2", 
                                                                                    "3":"C3"})



In [26]:



CLINPATH['path_ad_level'] = CLINPATH['path_ad_level'].replace({"No evidence": "No evidence of Alzheimer\'s disease neuropathological change"})





In [27]:
path_autopsy_map = { "Parkinson's disease with dementia": "Parkinson's disease with dementia", 
       "Parkinson's disease": "Parkinson's disease",
       'Control brain':"Control, no misfolded protein or significant vascular pathology", 
       'Pathological ageing': 'Control, no misfolded protein or significant vascular pathology',
       'Control brain / Path ageing': 'Control, no misfolded protein or significant vascular pathology',
       'Argyrophilic grain disease': "Control, Argyrophilic grain disease",
       'Control brain, Cerebrovascular disease (small vessel)':"Control, Cerebrovascular disease (atherosclerosis)",
       'Cerebrovascular disease (small vessel)':"Control, Cerebrovascular disease (atherosclerosis)",
       "Control brain, Alzheimer`s disease (intermediate level AD pathological change)":"Alzheimer's disease (intermediate level neuropathological change)",
       'Control brain / Path ageing, CAA':"Control, Cerebrovascular disease (cerebral amyloid angiopathy)"}


In [28]:
CLINPATH['path_autopsy_dx_main'] = CLINPATH['path_autopsy_dx_main'].replace(path_autopsy_map)

In [29]:
CLINPATH['path_autopsy_dx_main'].unique()

array(["Parkinson's disease with dementia", "Parkinson's disease",
       'Control, no misfolded protein or significant vascular pathology',
       'Control, Argyrophilic grain disease',
       'Control, Cerebrovascular disease (atherosclerosis)',
       "Alzheimer's disease (intermediate level neuropathological change)",
       'Control, Cerebrovascular disease (cerebral amyloid angiopathy)',
       'Nan'], dtype=object)

In [30]:
# remove any text nans
CLINPATH.replace("Nan", pd.NA, inplace=True)



In [31]:
CLINPATH['source_sample_id'] = pd.NA

# CLINPATH = reorder_table_to_CDE(CLINPATH, "CLINPATH", CDE_df)


added empty column source_sample_id to CLINPATH table


In [32]:

report = ReportCollector(destination="")
validate_table(CLINPATH, "CLINPATH", CDE_df, report)
report.print_log()




All required fields are present in *CLINPATH* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- age_at_onset: 138/138 empty rows

	- age_at_diagnosis: 10/138 empty rows

	- first_motor_symptom: 138/138 empty rows

	- path_year_death: 138/138 empty rows

	- brain_weight: 138/138 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- smoking_years: 138/138 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*hx_dementia_mci*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Yes', 'No'
- _*hx_melanoma*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Yes', 'No'
- _*education_level*_:  invalid values 💩'<NA>'
    - valid ➡️ 'High School', 'High School/GED', 'Some college without degree', 'Associate degree college', 'Bachelor's degree', 'Master's degree', 'Professional or doctoral degree', 'Refuse', 'Other'
- _*smoking_status*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Current smoker', 'Former smoker', 'Never', 'Unknown'
- _*APOE_e4_status*_:  invalid values 💩'<NA>'
    - valid ➡️ '22', '23

In [35]:


# # write the clean metadata
# STUDY.to_csv(data_path / "metadata/STUDY.csv")
# PROTOCOL.to_csv(data_path / "metadata/PROTOCOL.csv")
# CLINPATH.to_csv(data_path / "metadata/CLINPATH.csv")
# SAMPLE.to_csv(data_path / "metadata/SAMPLE.csv")
# SUBJECT.to_csv(data_path / "metadata/SUBJECT.csv")

# # also writh them to clean...
# 
#  

export_root = Path.cwd() / "clean/team-Hardy"
if not export_root.exists():
    export_root.mkdir(parents=True, exist_ok=True)


STUDY.to_csv( export_root / "STUDY.csv")
PROTOCOL.to_csv(export_root / "PROTOCOL.csv")
SAMPLE.to_csv(export_root / "SAMPLE.csv")
SUBJECT.to_csv(export_root / "SUBJECT.csv")
CLINPATH.to_csv(export_root / "CLINPATH.csv")



Make sure that they read back in codrrectly.

In [182]:
STUDY_ = read_meta_table(export_root / "STUDY.csv", dtypes_dict)
PROTOCOL_ = read_meta_table(export_root / "PROTOCOL.csv", dtypes_dict)
SAMPLE_ = read_meta_table(export_root / "SAMPLE.csv", dtypes_dict)
SUBJECT_ = read_meta_table(export_root / "SUBJECT.csv", dtypes_dict)
CLINPATH_ = read_meta_table(export_root / "CLINPATH.csv", dtypes_dict)


In [183]:
report = ReportCollector(destination="")
validate_table(CLINPATH_, "CLINPATH", CDE_df, report)
report.print_log()


🚨⚠️❗ **Missing Required Fields in CLINPATH: source_sample_id**
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- age_at_onset: 138/138 empty rows

	- age_at_diagnosis: 10/138 empty rows

	- first_motor_symptom: 138/138 empty rows

	- path_year_death: 138/138 empty rows

	- brain_weight: 138/138 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- smoking_years: 138/138 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*hx_dementia_mci*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Yes', 'No'
- _*hx_melanoma*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Yes', 'No'
- _*education_level*_:  invalid values 💩'<NA>'
    - valid ➡️ 'High School', 'High School/GED', 'Some college without degree', 'Associate degree college', 'Bachelor's degree', 'Master's degree', 'Professional or doctoral degree', 'Refuse', 'Other'
- _*smoking_status*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Current smoker', 'Former smoker', 'Never', 'Unknown'
- _*APOE_e4_status*_:  invalid values 💩'<NA>'
    - valid ➡️

In [184]:
report = ReportCollector(destination="")
validate_table(SUBJECT_, "SUBJECT", CDE_df, report)
report.print_log()

All required fields are present in *SUBJECT* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- duration_pmi: 1/64 empty rows
No empty entries (Nan) found in _Optional_ fields.
## Enums
🚨⚠️❗ **Invalid entries**
- _*race*_:  invalid values 💩'<NA>'
    - valid ➡️ 'American Indian or Alaska Native', 'Asian', 'White', 'Black or African American', 'Multi-racial', 'Native Hawaiian or Other Pacific Islander', 'Other', 'Unknown', 'Not Reported'
- _*ethnicity*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Hispanic or Latino', 'Not Hispanic or Latino', 'Unknown', 'Not Reported'



In [185]:
report = ReportCollector(destination="")
validate_table(SAMPLE_, "SAMPLE", CDE_df, report)
report.print_log()

🚨⚠️❗ **Missing Required Fields in SAMPLE: source_sample_id**
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- source_RIN: 3616/3616 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- pm_PH: 3616/3616 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*sequencing_length*_:  invalid values 💩'190'
    - valid ➡️ '25', '50', '100', '150'



In [187]:
CLINPATH_.head()

,sample_id,source_subject_id,time_from_baseline,GP2_id,hemisphere,region_level_1,region_level_2,region_level_3,AMPPD_id,family_history,...,path_nia_ri,path_nia_aa_a,path_nia_aa_b,path_nia_aa_c,TDP43,arteriolosclerosis_severity_scale,amyloid_angiopathy_severity_scale,path_ad_level,dig_slide_avail,quant_path_avail
0,Libat_IPL,P27/11,0,MDGAP-QSBB_000088_s1,Left,Parietal lobe,Inferior parietal lobule,Grey matter,<NA>,Not Reported,...,<NA>,A1,B1,C0,<NA>,<NA>,<NA>,Low level Alzheimer's disease neuropathologica...,Yes,Yes
1,Libat_ACG,P27/11,0,MDGAP-QSBB_000088_s1,Left,Cingulate gyrus,Anterior cingulate gyrus,Grey matter,<NA>,Not Reported,...,<NA>,A1,B1,C0,<NA>,<NA>,<NA>,Low level Alzheimer's disease neuropathologica...,Yes,Yes
2,Rijof_IPL,P8/18,0,MDGAP-QSBB_000583_s1,Right,Parietal lobe,Inferior parietal lobule,Grey matter,<NA>,Not Reported,...,<NA>,A1,B1,C0,<NA>,<NA>,<NA>,Low level Alzheimer's disease neuropathologica...,Yes,Yes
3,Rijof_ACG,P8/18,0,MDGAP-QSBB_000583_s1,Right,Cingulate gyrus,Anterior cingulate gyrus,Grey matter,<NA>,Not Reported,...,<NA>,A1,B1,C0,<NA>,<NA>,<NA>,Low level Alzheimer's disease neuropathologica...,Yes,Yes
4,Gotar_IPL,P41/09,0,MDGAP-QSBB_000406_s1,Right,Parietal lobe,Inferior parietal lobule,Grey matter,<NA>,Not Reported,...,<NA>,A1,B1,C0,<NA>,<NA>,<NA>,Low level Alzheimer's disease neuropathologica...,Yes,Yes


## Team-Jacobsson

Look at the v1 versions of Team-Jacobsson... they mostly seem okay.  

TODO: clean them up and use to cross-validate the `update_schema` tools.

In [39]:
metadata_version = "v1"

CDE_df,dtypes_dict = read_ASAP_CDE(metadata_version=metadata_version)
METADATA_VERSION_DATE = f"{metadata_version}_{pd.Timestamp.now().strftime('%Y%m%d')}"


## convert 
data_path = Path.home() / ("Projects/ASAP/team-jakobsson")
metadata_path = data_path / "metadata_v1"

SUBJECT = pd.read_csv(f"{metadata_path}/SUBJECT.csv")
CLINPATH = pd.read_csv(f"{metadata_path}/CLINPATH.csv")
STUDY = pd.read_csv(f"{metadata_path}/STUDY.csv")
PROTOCOL = pd.read_csv(f"{metadata_path}/PROTOCOL.csv")
SAMPLE = pd.read_csv(f"{metadata_path}/SAMPLE.csv")


read url


In [40]:

report = ReportCollector(destination="")
validate_table(STUDY, "STUDY", CDE_df, report)
report.print_log()


🚨⚠️❗ **Missing Required Fields in STUDY: submitter_email**
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in STUDY. 🥳



In [42]:

STUDY['metadata_version_date'] = METADATA_VERSION_DATE
STUDY.rename({'submittor_email':'submitter_email'}, axis=1, inplace=True)
STUDY.head()

,project_name,project_dataset,project_description,ASAP_team_name,ASAP_lab_name,PI_full_name,PI_email,contributor_names,submitter_name,submitter_email,...,other_funding_source,publication_DOI,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,metadata_version_date
0,Activation of transposable elements as a trigg...,Single nuclei sequencing of brain regions from...,"A range of genetic, clinical and pathological ...",TEAM-JAKOBSSON,Gale-Hammell Lab,"Molly, Gale Hammell",Molly.GaleHammell@nyulangone.org,"Anita, Adami; Talitha, Forcier; Raquel, Garza;...","Oliver, H, Tam",oliver.tam@nyulangone.org,...,NaN,NaN,NaN,71,"Substantia nigra, prefrontal cortex, amygdala,...",Late stage PD & control,NaN,NaN,Creative Commons Attribution license CC BY 4.0,v1_20231114


In [43]:

report = ReportCollector(destination="")
validate_table(PROTOCOL, "PROTOCOL", CDE_df, report)
report.print_log()


All required fields are present in *PROTOCOL* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in PROTOCOL. 🥳



In [44]:

report = ReportCollector(destination="")
validate_table(SUBJECT, "SUBJECT", CDE_df, report)
report.print_log()


All required fields are present in *SUBJECT* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- duration_pmi: 7/25 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- primary_diagnosis_text: 25/25 empty rows
## Enum fields have valid values in SUBJECT. 🥳



In [45]:


SAMPLE = reorder_table_to_CDE(SAMPLE, "SAMPLE", CDE_df)
    
report = ReportCollector(destination="")
validate_table(SAMPLE, "SAMPLE", CDE_df, report)
report.print_log()

added empty column DV200 to SAMPLE table
All required fields are present in *SAMPLE* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- source_RIN: 339/339 empty rows

	- RIN: 339/339 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- DV200: 339/339 empty rows

	- pm_PH: 339/339 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*sequencing_length*_:  invalid values 💩'28', '91', '90', '8', '10'
    - valid ➡️ '25', '50', '100', '150'



In [46]:
CLINPATH = reorder_table_to_CDE(CLINPATH, "CLINPATH", CDE_df)

report = ReportCollector(destination="")
validate_table(CLINPATH, "CLINPATH", CDE_df, report)
report.print_log()



added empty column dig_slide_avail to CLINPATH table
added empty column quant_path_avail to CLINPATH table
All required fields are present in *CLINPATH* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- age_at_onset: 71/71 empty rows

	- age_at_diagnosis: 47/71 empty rows

	- first_motor_symptom: 71/71 empty rows

	- path_year_death: 71/71 empty rows

	- brain_weight: 71/71 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- smoking_years: 71/71 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*region_level_2*_:  invalid values 💩'Prefrontal cortex'
    - valid ➡️ 'Superior frontal gyrus', 'Middle frontal gyrus', 'Inferior frontal gyrus', 'Superior temporal gyrus', 'Middle temporal gyrus', 'Inferior temporal gyrus', 'Fusiform gyrus', 'Transentorhinal region', 'Entorinal region', 'Subiculum', 'CA1-CA4', 'Amygdala', 'Periamygdala cortex', 'Anterior cingulate gyrus', 'Posterior cingulate gyrus', 'Superior parietal lobule', 'Inferior parietal lobule', 'Parastriate cor

In [48]:

export_root = Path.cwd() / "clean/team-Jacobsson"
if not export_root.exists():
    export_root.mkdir(parents=True, exist_ok=True)


STUDY.to_csv( export_root / "STUDY.csv")
PROTOCOL.to_csv(export_root / "PROTOCOL.csv")
SAMPLE.to_csv(export_root / "SAMPLE.csv")
SUBJECT.to_csv(export_root / "SUBJECT.csv")
CLINPATH.to_csv(export_root / "CLINPATH.csv")


Transfer the data to our buckets